In [21]:
import numpy as np
import pandas as pd
import os
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import tensorflow_model_optimization as tfmot
from tensorflow import keras
import tempfile
import zipfile


In [22]:
# 모델 크기 계산 함수 
def get_gzipped_model_size(file):
  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)
  return os.path.getsize(zipped_file)

In [31]:
# 데이터 준비 및 전처리 (X는 특성, y는 점수)
X, y = load_wine(return_X_y=True)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [32]:
model = tf.keras.models.load_model("my_model_3.h5")
_, accuracy = model.evaluate(
   X_test, y_test, verbose=0)

In [33]:
# 모델 가충치 가지치기 기능 실행
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
batch_size = 128
epochs = 4
validation_split = 0.1
num_images = X_train.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}
model_for_pruning = prune_low_magnitude(model, **pruning_params)
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
logdir = tempfile.mkdtemp()
callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]
model_for_pruning.fit(X_train, y_train,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   X_test, y_test, verbose=0)
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
model_for_pruning.summary()

Epoch 1/4


c:\Users\Jay\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1/1 [==============================] - 2s 2s/step - loss: 1.0985 - accuracy: 0.3228 - val_loss: 1.0943 - val_accuracy: 0.4667
Epoch 2/4
1/1 [==============================] - 0s 43ms/step - loss: 1.0904 - accuracy: 0.7953 - val_loss: 1.0895 - val_accuracy: 0.6667
Epoch 3/4
1/1 [==============================] - 0s 39ms/step - loss: 1.0832 - accuracy: 0.8346 - val_loss: 1.0844 - val_accuracy: 0.6667
Epoch 4/4
1/1 [==============================] - 0s 42ms/step - loss: 1.0760 - accuracy: 0.8583 - val_loss: 1.0787 - val_accuracy: 0.6000
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_dense_  (None, 128)               3458      
 25 (PruneLowMagnitude)                                          
                                                                 
 prune_low_magnitude_dense_  (None, 64)                16450     
 26 (PruneLowMagnitude)                   

In [34]:
# 가중치 가지치기 기능 적용 모델 저장
model_for_pruning.save("purned_model_3.h5")

c:\Users\Jay\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [35]:
# 모델 양자화 기능 실행
quantize_model = tfmot.quantization.keras.quantize_model
q_aware_model = quantize_model(model)
q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
q_aware_model.fit(X_train, y_train,
                  batch_size=500, epochs=4, validation_split=0.1)
_, q_aware_model_accuracy = q_aware_model.evaluate(
   X_test, y_test, verbose=0)
q_aware_model.summary()

Epoch 1/4
1/1 [==============================] - 1s 1s/step - loss: 1.0670 - accuracy: 0.8346 - val_loss: 1.0785 - val_accuracy: 0.5333
Epoch 2/4
1/1 [==============================] - 0s 29ms/step - loss: 1.0554 - accuracy: 0.8898 - val_loss: 1.0645 - val_accuracy: 0.6667
Epoch 3/4
1/1 [==============================] - 0s 31ms/step - loss: 1.0448 - accuracy: 0.9370 - val_loss: 1.0528 - val_accuracy: 0.7333
Epoch 4/4
1/1 [==============================] - 0s 30ms/step - loss: 1.0296 - accuracy: 0.9055 - val_loss: 1.0411 - val_accuracy: 0.8000
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_3 (Quantize  (None, 13)                3         
 Layer)                                                          
                                                                 
 quant_dense_25 (QuantizeWr  (None, 128)               1797      
 apperV2)                       

In [36]:
# 양자화 기능 적용 모델 저장
q_aware_model.save("quantized_model_3.h5")

In [37]:
# 기능 매트릭 정리 
# 파일 크기를 정수로 변환하여 출력
keras_file_size = int(get_gzipped_model_size("my_model_3.h5"))
pruned_keras_file_size = int(get_gzipped_model_size("purned_model_3.h5"))
quant_file_size = int(get_gzipped_model_size("quantized_model_3.h5"))
# 압축률 = (압축 전 데이터 크기) / (압축 후 데이터 크기)
pruned_keras_compression_rate = ((keras_file_size / pruned_keras_file_size))
quant_file_compression_rate = (keras_file_size / quant_file_size)
pruned_keras_compression_rate = "{:.2f}".format(pruned_keras_compression_rate)
quant_file_compression_rate = "{:.2f}".format(quant_file_compression_rate)
# 원본 모델 비교
PADP = (accuracy - (accuracy - model_for_pruning_accuracy)) / accuracy * 100
if PADP >= 100:
    PADP = 100
P = "{:.2f}".format(PADP)
PADQ = (accuracy - (accuracy - q_aware_model_accuracy)) / accuracy * 100
if PADQ >= 100:
    PADQ = 100
Q = "{:.2f}".format(PADQ)

In [38]:
# 각 모델간(원본, 가중치 가지치기, 양자화) 성능비교 매트릭 출력
print('모델 사이즈---------------------------------------------------')
print("Size of gzipped baseline Keras model: %d bytes" % keras_file_size)
print("Size of gzipped pruned Keras model: %d bytes" % pruned_keras_file_size)
print("Size of gzipped Quantized model: %d bytes" % quant_file_size)
print('모델 압축률---------------------------------------------------')
print("compression rate of pruned Keras model: ", pruned_keras_compression_rate)
print("compression rate of Quantized model: ", quant_file_compression_rate)
print('모델 최적화 성능----------------------------------------------')
print('Baseline test accuracy:', accuracy)
print('Pruned test accuracy:', model_for_pruning_accuracy)
print('Quant test accuracy:', q_aware_model_accuracy)
print('모델 최적화 비교 정확도----------------------------------------')
print('가지치기 비교 정확도 :', P,"%")
print('양자화 비교 정확도 :', Q,"%")

모델 사이즈---------------------------------------------------
Size of gzipped baseline Keras model: 202180 bytes
Size of gzipped pruned Keras model: 198171 bytes
Size of gzipped Quantized model: 197420 bytes
모델 압축률---------------------------------------------------
compression rate of pruned Keras model:  1.02
compression rate of Quantized model:  1.02
모델 최적화 성능----------------------------------------------
Baseline test accuracy: 0.7592591643333435
Pruned test accuracy: 0.8333333134651184
Quant test accuracy: 0.9166666865348816
모델 최적화 비교 정확도----------------------------------------
가지치기 비교 정확도 : 100.00 %
양자화 비교 정확도 : 100.00 %
